In [43]:
#
# Update package history based on latest available statistics from ecosyste.ms and anaconda
#
# (C) Open Energy Transition (OET)
# Distributed via MIT license
#

# define the path of the CSV file listing the packages to assess
url = 'https://raw.githubusercontent.com/open-energy-transition/open-esd-analysis/main/esd_list.csv'


# no need to change something below ---------------------------
import os.path
import json
import io
from datetime import datetime, timedelta

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

try:
    from urllib.request import urlopen
except:
    !pip install urllib
    from urllib.request import urlopen
try:
    import requests
except:
    !pip install requests
    import requests

try:
    import itables
    from itables import init_notebook_mode
    from itables import to_html_datatable
except:
    !pip install itables
    import itables
    from itables import init_notebook_mode
    from itables import to_html_datatable

# setup variables
names = []
urls = []
descriptions = []
categories = []
sub_categories = []
languages = []
licenses = []
download_counts = []
total_dependent_repos_counts = []
stars = []
dois = []
citations = []
forks = []
contributors = []
develop_distr_scores = []
past_year_issues_counts = []
creates = []
updates = []

# get CSV file
df_packages = pd.read_csv(url,index_col=0)

# loop through packages mentioned in the CSV file
for index, row in df_packages.iterrows():
    #ä get the JSON file from the ost.ecosyste.ms
    json_url = row.iloc[0]
    r = requests.get(json_url)
    json_data = r.json()

    # store necessary details
    names.append(json_data['name'])
    urls.append(json_data['url'])
    descriptions.append(json_data['description'])
    categories.append(json_data['category'])
    sub_categories.append(json_data['sub_category'])
    languages.append(json_data['language'])
    licenses.append(json_data['repository']['license'])
    download_counts.append(0)
    total_dependent_repos_counts.append(0)
    stars.append(json_data['repository']['stargazers_count'])
    dois.append(0)
    citations.append(json_data['total_citations'])
    forks.append(json_data['repository']['forks_count'])
    contributors.append(json_data['repository']['commit_stats']['total_committers'])
    develop_distr_scores.append(json_data['commits']['dds'])
    past_year_issues_counts.append(json_data['issues_stats']['past_year_issues_count'])
    creates.append(datetime.strptime(json_data['repository']['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y/%m'))
    updates.append(datetime.strptime(json_data['repository']['updated_at'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y/%m'))

# prepare summary dataframe
df_extract = pd.DataFrame()
df_extract['Project Name'] = names
df_extract['Project Name'] = names
df_extract['Category'] = categories
df_extract['Sub Category'] = sub_categories
df_extract['Created'] = creates
df_extract['Updated'] = updates
df_extract['License'] = licenses
df_extract['Language'] = languages
df_extract['Citations'] = citations
df_extract['Stars'] = stars
df_extract['Contributors'] = contributors
df_extract['DDS'] = develop_distr_scores
df_extract['Forks'] = forks
df_extract['Dependents'] = total_dependent_repos_counts
df_extract['PM Downloads'] = download_counts
df_extract['PY Issues']= past_year_issues_counts

# show summary dataframe
print (df_extract)

            Project Name        Category                       Sub Category  \
0             pandapower  Energy Systems  Energy System Modeling Frameworks   
1                  PyPSA  Energy Systems  Energy System Modeling Frameworks   
2              PyPowSyBl  Energy Systems  Energy System Modeling Frameworks   
3            oemof-solph  Energy Systems  Energy System Modeling Frameworks   
4              openTEPES  Energy Systems  Energy System Modeling Frameworks   
5                   FINE  Energy Systems  Energy System Modeling Frameworks   
6               Minpower  Energy Systems  Energy System Modeling Frameworks   
7               Calliope  Energy Systems  Energy System Modeling Frameworks   
8               GridPath  Energy Systems  Energy System Modeling Frameworks   
9        PowerSystems.jl  Energy Systems  Energy System Modeling Frameworks   
10                  GenX  Energy Systems  Energy System Modeling Frameworks   
11           SpineOpt.jl  Energy Systems  Energy Sys

In [44]:
# adjust some details as they are not correct in ost.ecosyste.ms
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Antares Simulator'].values[0], 'License'] = 'mpl-2.0'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'FINE'].values[0], 'License'] = 'mit'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Minpower'].values[0], 'License'] = 'mit'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'pandapower'].values[0], 'License'] = 'bsd-3-clause'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'switch-model'].values[0], 'License'] = 'apache-2.0'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Temoa'].values[0], 'Language'] = 'Python'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'PyPowSyBl'].values[0], 'Language'] = 'Python'

df_extract.drop(columns=['Category', 'License', 'Sub Category', 'Owner', 'Repository', 'Language'], axis=1, errors='ignore', inplace=True)
df_extract.style.format({
    'Citations':'{:,.0f}',
    'Stars':'{:,.0f}',
    'DDS':'{:,.2f}',
    'PM Downloads':'{:,.0f}',
    'PY Issues':'{:,.0f}'
})
itables.show(
    # df_extract.loc[:, df_extract.columns != 'Repository'],
    df_extract,
    layout={"top1": "searchBuilder"},
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"],
    searchBuilder={},
    lengthMenu=[25, 50],
    order=[[0, "asc"]]
)

# Remark:
#   DDS ... development distribution score (the smaller the number the better; but 0 means no data available)
#   PM .. previous month (0 means either no downloads or not tracked/shared from the repository owner)
#   PY .. previous year (0 means either no issues or not tracked/shared from the repository owner)